# ラボ 5: Human in the Loop

## セットアップと初期化

まず、環境をセットアップし、必要なモジュールをインポートします。重要なコンポーネントはチェックポイントです:

```python
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()
```

チェックポイントは、Git などの分散バージョン管理システムと同様に機能しますが、AI の状態を対象としています。これにより、意思決定プロセスのさまざまな時点で AI の状態の「コミット」を作成し、必要に応じて「分岐」または「元に戻す」ことができます。

チェックポイントを過度に使用すると、特に状態オブジェクトが大きい場合は、メモリ使用量が大幅に増加する可能性があることに注意してください。長時間実行されるアプリケーションでは、古いチェックポイントを管理または期限切れにする戦略を実装します。それらのチェックポイントが不要になった場合は、データベースに実装するチェックポイントの Time-To-Live の使用を検討してください。
チェックポイント用のデータベースとして SQLite を使用しますが、実稼働アプリケーションでは Redis または Postgres に切り替えることもできます。

In [ ]:
from dotenv import load_dotenv
import os
import sys
import json, re
import pprint
import boto3
from botocore.client import Config
import warnings

warnings.filterwarnings("ignore")
import logging

# import local modules
dir_current = os.path.abspath("")
dir_parent = os.path.dirname(dir_current)
if dir_parent not in sys.path:
    sys.path.append(dir_parent)
from utils import utils

bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Set basic configs
logger = utils.set_logger()
pp = utils.set_pretty_printer()

# Load environment variables from .env file or Secret Manager
_ = load_dotenv("../.env")
aws_region = os.getenv("AWS_REGION")
tavily_ai_api_key = utils.get_tavily_api("TAVILY_API_KEY", aws_region)

# Set bedrock configs
bedrock_config = Config(
    connect_timeout=120, read_timeout=120, retries={"max_attempts": 0}
)

# Create a bedrock runtime client
bedrock_rt = boto3.client(
    "bedrock-runtime", region_name=aws_region, config=bedrock_config
)

# Create a bedrock client to check available models
bedrock = boto3.client("bedrock", region_name=aws_region, config=bedrock_config)


In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, ToolMessage
from langchain_aws import ChatBedrockConverse
from langchain_community.tools.tavily_search import TavilySearchResults
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

次に、カスタム メッセージ処理関数を使用してエージェントの状態を設定します。

In [ ]:
from uuid import uuid4
from langchain_core.messages import AnyMessage, SystemMessage, HumanMessage, AIMessage

"""
In previous examples we've annotated the `messages` state key
with the default `operator.add` or `+` reducer, which always
appends new messages to the end of the existing messages array.

Now, to support replacing existing messages, we annotate the
`messages` key with a customer reducer function, which replaces
messages with the same `id`, and appends them otherwise.
"""


def reduce_messages(
    left: list[AnyMessage], right: list[AnyMessage]
) -> list[AnyMessage]:
    # assign ids to messages that don't have them
    for message in right:
        if not message.id:
            message.id = str(uuid4())
    # merge the new messages with the existing messages
    merged = left.copy()
    for message in right:
        for i, existing in enumerate(merged):
            # replace any existing messages with the same id
            if existing.id == message.id:
                merged[i] = message
                break
        else:
            # append any new messages to the end
            merged.append(message)
    return merged


class AgentState(TypedDict):
    messages: Annotated[list[AnyMessage], reduce_messages]

この機能は、マイクロサービス アーキテクチャのスマート メッセージ ブローカーのように機能します。これにより、会話履歴が一貫性を保ち、最新の状態に保たれます。これは、長時間実行される AI 対話でコンテキストを維持するために重要です。

## ツールとエージェントのセットアップ

Tavily 検索ツールを統合します。

In [ ]:
tool = TavilySearchResults(max_results=2)

## エージェント クラスにおける人間の介入

`Agent` クラスは、AI の意思決定プロセスへの人間の介入を可能にするいくつかの重要な機能を実装します。

1. **アクションの前に中断する**:

   ```python
   self.graph = graph.compile(
       checkpointer=checkpointer, interrupt_before=["action"]
   )
   ```

   この行は、「アクション」ノードの前に割り込みを設定します。これにより、アクションが実行される前に人間が監視できるようになり、確認や変更の機会が提供されます。
   次のラボで説明するように、任意のノードの前に割り込みを追加できます。

2. **状態の検査**:
   `exists_action` メソッドは現在の状態を出力します。

   ```python
   def exists_action(self, state: AgentState):
       print(state)
       # ...
   ```

   これにより、人間は AI の推論や意図された動作など、AI の現在の状態を検査できるようになります。

3. **アクションの可視性**:
   `take_action` メソッドでは、各ツール呼び出しが出力されます。

   ```python
   print(f"Calling: {t}")
   ```

   これにより、AI がこれから実行するアクションが可視化され、人間による介入が可能になります。

4. **チェックポイント**:
   コンストラクタの `checkpointer` パラメータを使用すると、状態を保存および読み込むことができます。これにより、「タイムトラベル」機能が有効になり、人間が以前の決定ポイントを再訪して変更できるようになります。

5. **変更可能な状態**:
   `AgentState` は変更可能な構造です。このクラスでは直接示されていませんが、状態の変更が可能であり、人間が AI のコンテキストや決定を変更できるようになります。

これらの機能が総合的に、人間が関与する AI のフレームワークを構築し、重要な局面で人間のオペレーターが AI の意思決定プロセスを監視、介入、ガイドできるようになります。

In [ ]:
class Agent:
    def __init__(self, model, tools, system="", checkpointer=None):
        self.system = system
        graph = StateGraph(AgentState)
        graph.add_node("llm", self.call_bedrock)
        graph.add_node("action", self.take_action)
        graph.add_conditional_edges(
            "llm", self.exists_action, {True: "action", False: END}
        )
        graph.add_edge("action", "llm")
        graph.set_entry_point("llm")
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
        self.tools = {t.name: t for t in tools}
        self.model = model.bind_tools(tools)

    def call_bedrock(self, state: AgentState):
        messages = state["messages"]
        if self.system:
            messages = [SystemMessage(content=self.system)] + messages
        message = self.model.invoke(messages)
        return {"messages": [message]}

    def exists_action(self, state: AgentState):
        print(state)
        result = state["messages"][-1]
        return len(result.tool_calls) > 0

    def take_action(self, state: AgentState):
        tool_calls = state["messages"][-1].tool_calls
        results = []
        for t in tool_calls:
            print(f"Calling: {t}")
            result = self.tools[t["name"]].invoke(t["args"])
            results.append(
                ToolMessage(tool_call_id=t["id"], name=t["name"], content=str(result))
            )
        print("Back to the model!")
        return {"messages": results}

`interrupt_before=["action"]` パラメータは、AI パイプラインの重要な制御ポイントを実装します。

これは、CI/CD パイプラインに承認ゲートを実装するのと似ており、必要なチェックなしで重要なアクションが実行されないようにします。

AI の安全性と制御メカニズムの理解を深めるには、2016 年に Google、スタンフォード、バークレーの研究者が発表した論文 ["AI の安全性における具体的な問題"](https://arxiv.org/abs/1606.06565) を参照してください。

## エージェントの初期化と実行

Amazon Bedrock で Claude を使用してエージェントを初期化します。

In [ ]:
prompt = """You are a smart research assistant. Use the search engine to look up information. \
You are allowed to make multiple calls (either together or in sequence). \
Only look up information when you are sure of what you want. \
If you need to look up some information before asking a follow up question, you are allowed to do that!
"""
model = ChatBedrockConverse(
    client=bedrock_rt,
    model="anthropic.claude-3-haiku-20240307-v1:0",
    temperature=0,
    max_tokens=None,
)
abot = Agent(model, [tool], system=prompt, checkpointer=memory)

> 追記　プロンプトの翻訳

```
あなたは賢いリサーチ アシスタントです。検索エンジンを使用して情報を検索してください。\
複数の通話 (同時にまたは連続して) を行うことができます。\
必要な情報がわかっている場合にのみ情報を検索してください。\
フォローアップの質問をする前に情報を検索する必要がある場合は、検索することができます。
```

In [ ]:
messages = [HumanMessage(content="Whats the weather in Berlin?")]
thread = {"configurable": {"thread_id": "1"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
thread

## 状態の検査と変更

エージェントの状態を検査および変更する機能は強力な機能です。

In [ ]:
abot.graph.get_state(thread)

この機能は、実稼働環境にライブ デバッガーを設置するのと似ており、実行中のプロセスの状態を検査および変更できます。

In [ ]:
abot.graph.get_state(thread).next

ご覧のとおり、次に実行されるノードは `('action',)` ノードです。

`action` ノードの前に割り込みセットを使用してグラフをコンパイルしたことを思い出してください。

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```


### ...割り込み後に続行

これでアクション (tavily-ai の呼び出し) の前に停止したので、続行できます。どのように動作するか見てみましょう。

In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
abot.graph.get_state(thread).next

### 人間による入力の追加

次に、Web を検索する `action` を実行するための承認を追加する方法を説明します。

JupyterLab で実行しているか、VS Code などで実行しているかに応じて、入力ボックスがコード セルの下または上部に表示されます。

![ユーザー入力を求める入力ボックスの例](../assets/lab5_1.png)

検索対象に問題がない場合は、はいを表す `y` を追加してください。それ以外の場合は、操作が中止されます。

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "2"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

while abot.graph.get_state(thread).next:
    print("\n", abot.graph.get_state(thread), "\n")
    _input = input("proceed?")
    if _input != "y":
        print("aborting")
        break
    for event in abot.graph.stream(None, thread):
        for v in event.values():
            print(v)

ご覧のとおり、次に実行されるノードは `('action',)` ノードです。

`action` ノードの前に割り込みセットを使用してグラフをコンパイルしたことを思い出してください。

```python
        self.graph = graph.compile(
            checkpointer=checkpointer, interrupt_before=["action"]
        )
```

---


ここで非常に興味深いことの 1 つは、すべてのツールではなく、特定のツール セットを呼び出すときにのみ停止して人間の入力を求めることです。

読み進める前に、これをどのように実現できるか考えてみてください。

### 特定のツール呼び出しでのみ停止

1. ツール呼び出しを解析し、ツール呼び出しパラメータ `name` (ツール名) が停止パラメータと同じ場合にのみ実行を停止します。

2. 停止する必要があるすべてのツールを追加ノードに追加します。

一般的に、オプション 2 の方がわかりやすく、デバッグも簡単です。 より大きな例でこれがどのように実装されているかを確認したい場合は、LangGraph の例の [カスタマー サポート エージェント](https://langchain-ai.github.io/langgraph/tutorials/customer-support/customer-support/#part-3-conditional-interrupt) にアクセスして、機密性の高いツールがどのように処理されるかを確認してください。

グラフのプレビューを以下に示します。

![機密性の高いツールと安全なツールを含むカスタマー サポート エージェント グラフ](../assets/lab5_2.png)

## 状態の変更

割り込みが発生するまで実行し、その後状態を変更します。

In [ ]:
messages = [HumanMessage("Whats the weather in LA?")]
thread = {"configurable": {"thread_id": "3"}}
for event in abot.graph.stream({"messages": messages}, thread):
    for v in event.values():
        print(v)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
current_values = abot.graph.get_state(thread)

In [ ]:
current_values.values["messages"][-1]

In [ ]:
current_values.values["messages"][-1].tool_calls

In [ ]:
_id = current_values.values["messages"][-1].tool_calls[0]["id"]
current_values.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in Munich"},
        "id": _id,
    }
]

In [ ]:
abot.graph.update_state(thread, current_values.values)

In [ ]:
abot.graph.get_state(thread)

In [ ]:
for event in abot.graph.stream(None, thread):
    for v in event.values():
        print(v)

## タイムトラベル

In [ ]:
states = []
for state in abot.graph.get_state_history(thread):
    print(state)
    print("--")
    states.append(state)

撮影されたのと同じ状態を取得するには、以下のオフセットを `-1` から `-3` に変更します。これにより、最新バージョンのソフトウェアで状態メモリに保存される初期状態 `__start__` と最初の状態が考慮されています。

In [ ]:
to_replay = states[-3]

In [ ]:
to_replay

In [ ]:
for event in abot.graph.stream(None, to_replay.config):
    for k, v in event.items():
        print(v)

## 時間を遡って編集する

In [ ]:
to_replay

In [ ]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]
to_replay.values["messages"][-1].tool_calls = [
    {
        "name": "tavily_search_results_json",
        "args": {"query": "current weather in LA, accuweather"},
        "id": _id,
    }
]

In [ ]:
branch_state = abot.graph.update_state(to_replay.config, to_replay.values)

In [ ]:
for event in abot.graph.stream(None, branch_state):
    for k, v in event.items():
        if k != "__end__":
            print(v)

## 指定された時間に状態にメッセージを追加する

In [ ]:
to_replay

In [ ]:
_id = to_replay.values["messages"][-1].tool_calls[0]["id"]

In [ ]:
# Lets update the humidity to something that is impossible

state_update = {
    "messages": [
        ToolMessage(
            tool_call_id=_id,
            name="tavily_search_results_json",
            content="23 degree celcius, 110 percent humidity",
        )
    ]
}

In [ ]:
branch_and_add = abot.graph.update_state(
    to_replay.config, state_update, as_node="action"
)

In [ ]:
for event in abot.graph.stream(None, branch_and_add):
    for k, v in event.items():
        print(v)

# 追加練習

## 小さなグラフを作成する

これは、状態メモリの制御についてさらに詳しく知りたい場合に調整できる、小さくてシンプルなグラフです。

In [ ]:
from dotenv import load_dotenv

_ = load_dotenv()

In [ ]:
from langgraph.graph import StateGraph, END
from typing import TypedDict, Annotated
import operator

次の状態を持つ単純な 2 ノード グラフを定義します。 -`lnode`: 最後のノード -`scratch`: スクラッチパッドの場所 -`count`: 各ステップで増加するカウンター

In [ ]:
class AgentState(TypedDict):
    lnode: str
    scratch: str
    count: Annotated[int, operator.add]

In [ ]:
def node1(state: AgentState):
    print(f"node1, count:{state['count']}")
    return {
        "lnode": "node_1",
        "count": 1,
    }


def node2(state: AgentState):
    print(f"node2, count:{state['count']}")
    return {
        "lnode": "node_2",
        "count": 1,
    }

グラフは N1->N2->N1... と進みますが、カウントが 3 に達すると途切れます。

In [ ]:
def should_continue(state):
    return state["count"] < 3

In [ ]:
builder = StateGraph(AgentState)
builder.add_node("Node1", node1)
builder.add_node("Node2", node2)

builder.add_edge("Node1", "Node2")
builder.add_conditional_edges("Node2", should_continue, {True: "Node1", False: END})
builder.set_entry_point("Node1")

In [ ]:
memory = MemorySaver()
graph = builder.compile(checkpointer=memory)

### 実行します!

では、スレッドを設定して実行します!

In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread)

### 現在の状態を確認する

現在の状態を取得します。AgentState である `values` に注意してください。`config` と `thread_ts` に注意してください。これらを使用して、以下のスナップショットを参照します。

In [ ]:
graph.get_state(thread)

メモリ内のすべてのステートスナップショットを表示します。表示される `count` エージェントステート変数を使用して、表示される内容を追跡できます。最新のスナップショットが最初にイテレータによって返されることに注意してください。また、メタデータには、グラフ実行のステップ数をカウントする便利な `step` 変数があることにも注意してください。これは少し詳細ですが、_parent_config_ が前のノードの _config_ であることにも気付くでしょう。最初の起動時に、追加のステートがメモリに挿入され、親が作成されます。これは、以下で分岐または _time travel_ するときに確認するものです。

### 状態の履歴を見る

In [ ]:
for state in graph.get_state_history(thread):
    print(state, "\n")

`config` のみをリストに保存します。右側のカウントの順序に注意してください。`get_state_history` は、最新のスナップショットを最初に返します。

In [ ]:
states = []
for state in graph.get_state_history(thread):
    states.append(state.config)
    print(state.config, state.values["count"])

初期の状態を取得します。

In [ ]:
states[-3]

これは、Node1 が初めて完了した後の状態です。`next` は `Node2` であり、`count` は 1 であることに注意してください。

In [ ]:
graph.get_state(states[-3])

### 時間を遡る

`invoke` でその状態を使用して時間を遡ります。states[-3] を _current_state_ として使用し、node2 に続くことに注意してください。

In [ ]:
graph.invoke(None, states[-3])

新しい状態が状態履歴に追加されたことに注目してください。右端のカウントに注目してください。

In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state.config, state.values["count"])

詳細は以下を参照してください。テキストはたくさんありますが、新しいブランチを開始するノードを見つけてください。親 _config_ はスタック内の前のエントリではなく、state[-3] からのエントリであることに注意してください。

In [ ]:
thread = {"configurable": {"thread_id": str(1)}}
for state in graph.get_state_history(thread):
    print(state, "\n")

### 状態の変更

まず、新しいスレッドを開始して実行し、履歴を消去してみましょう。

In [ ]:
thread2 = {"configurable": {"thread_id": str(2)}}
graph.invoke({"count": 0, "scratch": "hi"}, thread2)

In [ ]:
from IPython.display import Image

Image(graph.get_graph().draw_png())

In [ ]:
states2 = []
for state in graph.get_state_history(thread2):
    states2.append(state.config)
    print(state.config, state.values["count"])

まず状態を取得することから始めます。

In [ ]:
save_state = graph.get_state(states2[-3])
save_state

ここで値を変更します。注意すべき微妙な点が 1 つあります。エージェントの状態を定義したとき、`count` は `operator.add` を使用して、値が現在の値に _追加_ されることを示しました。ここでは、`-3` が現在の count 値を置き換えるのではなく、現在の count 値に追加されます。

In [ ]:
save_state.values["count"] = -3
save_state.values["scratch"] = "hello"
save_state

次に、状態を更新します。これにより、メモリ内の _top_、つまり _latest_ エントリに新しいエントリが作成されます。これが現在の状態になります。

In [ ]:
graph.update_state(thread2, save_state.values)

現在の状態は上部にあります。`thread_ts` を一致させることができます。
新しいノードの `parent_config`、`thread_ts` に注意してください。これは前のノードです。

In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

### `as_node` でもう一度試してください

`update_state()` を使用して書き込む場合、どのノードを書き込み元として想定するかをグラフ ロジックに定義する必要があります。これにより、グラフ ロジックがグラフ上のノードを見つけられるようになります。値を書き込んだ後、新しい状態を使用してグラフをトラバースして `next()` 値が計算されます。この場合、状態は `Node1` によって書き込まれました。グラフは、次の状態を `Node2` として計算できます。一部のグラフでは、条件付きエッジを通過する必要があることに注意してください。試してみましょう。

In [ ]:
graph.update_state(thread2, save_state.values, as_node="Node1")

In [ ]:
for i, state in enumerate(graph.get_state_history(thread2)):
    if i >= 3:  # print latest 3
        break
    print(state, "\n")

特定の `thread_ts` が指定されていない場合、 `invoke` は現在の状態から実行されます。これが今追加されたエントリです。

In [ ]:
graph.invoke(None, thread2)

状態履歴を出力し、最新のエントリの `scratch` 値の変更に注目してください。

In [ ]:
for state in graph.get_state_history(thread2):
    print(state, "\n")

実験を続けてください！